# Análise descritiva

Este componente realiza a análise descritiva de cada coluna do dataset fornecido. Para colunas numéricas, são calculadas as seguintes estatísticas: `count`, `mean`, `std`, `min`, `max`, além dos percentis. Para dados categóricos, por outro lado, calculam-se `count`, `unique`, `top` e `freq`. O valor de `top` representa a **moda** , enquanto `freq` é a frequência da **moda**.

## Declaração de parâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parameters
dataset = "/tmp/data/titanic.csv" #@param {type:"string"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd
from platiagro import stat_dataset

df = pd.read_csv(dataset)
featuretypes = stat_dataset(name=dataset)["featuretypes"]

## Realiza a análise descritiva

In [ ]:
import numpy as np
from platiagro import CATEGORICAL, NUMERICAL

for i, col in enumerate(df):
    if featuretypes[i] == CATEGORICAL:
        df[col] = df[col].astype("category")


def get_description(df, col_type):
    idx = np.where(np.array(featuretypes) == col_type)[0]
    if len(idx) > 0:
        return df.iloc[:, idx].describe().T
    return None


categorical_description = get_description(df, CATEGORICAL)
numeric_description = get_description(df, NUMERICAL)

## Salva as tabelas com as estatísticas da análise descritiva

In [ ]:
import matplotlib.pyplot as plt


def format_str_cell(text, max_len=20):
    return text[:max_len] + "..." if len(text) > max_len else text


def render_table(
    data,
    col_width=3.0,
    row_height=0.625,
    font_size=8,
    header_color="#40466e",
    row_colors=["#f1f1f2", "w"],
    edge_color="w",
    bbox=[0, 0, 1, 1],
    header_columns=0,
    ax=None,
    **kwargs
):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array(
            [col_width, row_height]
        )
        fig, ax = plt.subplots(figsize=size)
        ax.axis("off")

    # Format cells
    cellText = [
        [format_str_cell(j) if type(j) == str else "%.2f" % j for j in i]
        for i in data.values
    ]

    mpl_table = ax.table(
        cellText=cellText,
        bbox=bbox,
        colLabels=data.columns,
        rowLabels=[format_str_cell(x) for x in data.index],
        **kwargs
    )

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight="bold", color="w")
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0] % len(row_colors)])
    return ax


if categorical_description is not None:
    render_table(categorical_description, header_columns=0, col_width=2.0)
    plt.show()

if numeric_description is not None:
    render_table(numeric_description, header_columns=0, col_width=2.0)
    plt.show()